In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-11-04 17:27:49--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt'

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-11-04 17:27:49 (46.3 MB/s) - 'input.txt' saved [1115394/1115394]



In [2]:
with open('/kaggle/working/input.txt', 'r') as f:
    text = f.read()
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
stoi = {i: j for i, j in enumerate(chars)}
itos = {j: i for i, j in enumerate(chars)}

encode = lambda input: [itos[ch] for ch in input]
decode = lambda input: ''.join([stoi[ch] for ch in input])

In [5]:
print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [6]:
len(encode(text))

1115394

In [7]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size, 1))
    add_ix = torch.arange(1, block_size).repeat(batch_size, 1)
    final_ix = torch.cat((ix, ix + add_ix), dim=1)
    xb = data[final_ix]
    yb = data[final_ix +  1]
    return xb, yb

In [10]:
xb, yb= get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('------')

for i in range(batch_size): 
    for j in range(1, block_size + 1):
        print(f"When input is {xb[i, :j].tolist()} the target is {yb[i, j-1]}")
        print(f'When input is "{decode(xb[i, :j].tolist())}" the target is "{decode([yb[i, j-1].item()])}"')
    print()

inputs:
torch.Size([4, 8])
tensor([[58, 63,  8,  0,  0, 19, 24, 27],
        [39, 59, 45, 46, 58,  1, 46, 43],
        [49, 43, 57,  1, 53, 50, 42,  1],
        [52, 41, 47, 43, 52, 58,  1, 56]])
targets:
torch.Size([4, 8])
tensor([[63,  8,  0,  0, 19, 24, 27, 33],
        [59, 45, 46, 58,  1, 46, 43,  1],
        [43, 57,  1, 53, 50, 42,  1, 46],
        [41, 47, 43, 52, 58,  1, 56, 47]])
------
When input is [58] the target is 63
When input is "t" the target is "y"
When input is [58, 63] the target is 8
When input is "ty" the target is "."
When input is [58, 63, 8] the target is 0
When input is "ty." the target is "
"
When input is [58, 63, 8, 0] the target is 0
When input is "ty.
" the target is "
"
When input is [58, 63, 8, 0, 0] the target is 19
When input is "ty.

" the target is "G"
When input is [58, 63, 8, 0, 0, 19] the target is 24
When input is "ty.

G" the target is "L"
When input is [58, 63, 8, 0, 0, 19, 24] the target is 27
When input is "ty.

GL" the target is "O"
When i

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        
        logits = self.embed(idx)
        if targets is None:
            loss = None
        else:
            
            #logits shape (B, T), targets shape (B, T)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)

        return idx
            

In [18]:
bgram = BigramLanguageModel()
out, loss = bgram(xb, yb)
print(out.shape)
print(loss)
print(decode(bgram.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.8250, grad_fn=<NllLossBackward0>)

,jm-TFpfgoIZiJpwwjKTToBsu?&.HGnhoB;TMW'LGjj:&LEYqglbJHQPAuNI
H';DSHkb sajI.Saw!gkxPVG3;EQxY&$m?riqxP


In [19]:
optim = torch.optim.Adam(bgram.parameters(), lr=1e-3)

In [20]:
batch_size=32
for i in range(50000):
    xb, yb = get_batch('train')
    out, loss = bgram(xb, yb)
    optim.zero_grad()
    loss.backward()
    optim.step()
    
print(loss.item())

2.3840785026550293


In [21]:
print(decode(bgram.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


RIED teshaithed caveit t y oth d s?
ALoroe?
Wit rsthangbeeliuppeced d br owiuce yo if faizencangnso, se Gixthet, The t ghoorife iowinicowinde s ten, d s cand
OLINou focco INLII:
RKiveronomanr:
Ahe mavesedouk has;
WAllilld st prl blo y, m crd tl athe hoves tu w.
RENGlfed, sthatherep uedany tht boficof wieve ss OFotealiny kece tre, t ofre.
He l my, the steyowen ofre Br jurou habu ffiff angerpourdofe thas:

TEO,
I's; sperd.
Aw:
ifoulathyesot al mio st llllontorticas sin chin wethenconk'sty ay outhr


In [323]:
x = torch.randn((4, 8, 2))
B, T, C = x.shape

In [367]:
a = torch.tril(torch.ones((3,3)))
a = a / a.sum(dim=1, keepdim=True)
b = torch.randint(0, 10, (3,2), dtype=torch.float)
c = a @ b 
print('a=')
print(a)
print('----')
print('b=')
print(b)
print('----')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----
b=
tensor([[2., 7.],
        [5., 4.],
        [1., 0.]])
----
c=
tensor([[2.0000, 7.0000],
        [3.5000, 5.5000],
        [2.6667, 3.6667]])


In [325]:
xbow = torch.zeros_like(x)
for i in range(B):
    for j in range(T):
        xbow[i,j] = torch.mean(x[i, :j+1], 0)

print(x.shape)
print(x[0])
print(xbow[0])

torch.Size([4, 8, 2])
tensor([[ 0.5300,  0.5039],
        [ 0.9270,  1.2622],
        [-1.3136,  1.3034],
        [ 2.6133,  1.0745],
        [ 1.5550,  1.0246],
        [ 0.2637,  0.0498],
        [ 0.8311,  0.8405],
        [-0.5485,  0.6853]])
tensor([[0.5300, 0.5039],
        [0.7285, 0.8830],
        [0.0478, 1.0231],
        [0.6892, 1.0360],
        [0.8624, 1.0337],
        [0.7626, 0.8697],
        [0.7724, 0.8656],
        [0.6073, 0.8430]])


In [347]:
trils = torch.tril(torch.ones((T,T)))
trils = trils / trils.sum(dim=1, keepdim=True)
xbow2 = trils @ x
xbow2[0]

tensor([[0.5300, 0.5039],
        [0.7285, 0.8830],
        [0.0478, 1.0231],
        [0.6892, 1.0360],
        [0.8624, 1.0337],
        [0.7626, 0.8697],
        [0.7724, 0.8656],
        [0.6073, 0.8430]])

In [92]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_iters = 5000
eval_interval = 300
lr = 1e-3
eval_iters = 20
n_embd = 32


class BigramLanguageModel(nn.Module):
    def __init__(self): 
        super().__init__()
        self.embed = nn.Embedding(vocab_size, n_embd)
        self.lm_layer = nn.Linear(n_embd, vocab_size)
        self.pos_enc = nn.Embedding(block_size, n_embd)

    def forward(self, idx, targets=None):
        # idx shape (B, T)
        (B, T) = idx.shape
        token_embd = self.embed(idx)
        if targets is None:
            pos_embd = self.pos_enc(torch.tensor([0]))
        else:
            pos_embd = self.pos_enc(torch.arange(T))
            
        logits = self.lm_layer(token_embd + pos_embd)
        
        if targets is None:
            loss = None
        else:
            #logits shape (B, T), targets shape (B, T)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)

        return idx
            

In [93]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, targets=Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [94]:
model = BigramLanguageModel()
model = model.to(device)

In [103]:
model.pos_enc

Embedding(8, 32)

In [99]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr*0.1)

In [100]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, targets=yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.4677, val loss 2.5389
step 300: train loss 2.4508, val loss 2.5187
step 600: train loss 2.4568, val loss 2.5190
step 900: train loss 2.4863, val loss 2.5189
step 1200: train loss 2.4439, val loss 2.4987
step 1500: train loss 2.4531, val loss 2.4786
step 1800: train loss 2.4661, val loss 2.5306
step 2100: train loss 2.4668, val loss 2.5241
step 2400: train loss 2.4532, val loss 2.5048
step 2700: train loss 2.4578, val loss 2.5146
step 3000: train loss 2.4610, val loss 2.5165
step 3300: train loss 2.4582, val loss 2.4831
step 3600: train loss 2.5053, val loss 2.5433
step 3900: train loss 2.4429, val loss 2.5004
step 4200: train loss 2.4887, val loss 2.5411
step 4500: train loss 2.4733, val loss 2.5106
step 4800: train loss 2.4684, val loss 2.5137


In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

In [20]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.1 MB/s eta 0:00:0000:01


In [53]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
print(enc.n_vocab)
input = "oh sorryajh my bad!"
eng_char_tokens = []

#prove that tokenizer have to have a single charachter reprezentation
for char_idx in range(ord('A'), ord('Z') + 1):
    eng_char_tokens.append(enc.encode(chr(char_idx))[0])
for char_idx in range(ord('a'), ord('z') + 1):
    eng_char_tokens.append(enc.encode(chr(char_idx))[0])

assert 26 * 2 == len(eng_char_tokens)

print(eng_char_tokens)
print(enc.encode(input))
for token in enc.encode(input):
    print(enc.decode([token]))
print(enc.decode(enc.encode(input)))

50257
[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
[1219, 7926, 1228, 71, 616, 2089, 0]
oh
 sorry
aj
h
 my
 bad
!
oh sorryajh my bad!
